In [2]:
import pandas as pd
import pickle
from threading import Thread
import os

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
stemmer = SnowballStemmer("english")
cachedStopWords = stopwords.words("english")

In [3]:
##methods
            
def save_it_all(obj, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    
def load_objects(file):
    with open(file, 'rb') as input:
        return pickle.load(input)
    
def vocabDict(vocab, doc, n): # handles empty word sets
    nk = 0
    # n = the number of word postions for this document type
    probability = math.log(nk + 1) - math.log(n + len(vocab))

    newDict = {"count": 0, "probability":probability}
    vdict = dict((el,newDict) for el in vocab)
    return vdict

def saveFrame(df, name):
    df.to_csv(name+".csv", index=False, sep=",", header=True)
    save_it_all(df, name+".pkl")
    
def getMaxClass(dictionary):
    result = {"class": None, "max": None, "values": None}
    maxVal = max(dictionary.values())
    for key, value in dictionary.items():
        if maxVal == value:
            result["class"] = key
            result["max"] = value
            result["values"] = dictionary
    return result

# niave bayes caclulation 
def classDict(df):
    classificationDict = {}
    for docType in df["Type"]:
        classificationDict[docType] = None
    return classificationDict

def process(trainedFrame, row, index, classProb, vocab):
    for docType in trainedFrame["Type"]:
        #print(docType)
        wordProbs = 1
        for word in row["Document"].split():
            if word in vocab:
                #print("Word: "+word + ", Prob: " + str(dataTrained['wordCount'][dataTrained["Type"] == docType].tolist()[0][word]["probability"]))
                wordProbs = wordProbs * trainedFrame['WordCount'][trainedFrame["Type"] == docType].tolist()[0][word]["probability"]
        classProb[docType] = wordProbs * trainedFrame['ClassProbability'][trainedFrame["Type"] == docType].tolist()[0]    
    result = getMaxClass(classProb)
    # resultFrame["Predicted"].iloc[indexVal] 
    resultFrame.at[index, 'Predicted'] = result
    
    
    

In [4]:
# # load test data 
# # load in the test data
# fileName = "../forumTest.data"

# # main:
# data = read(fileName)
# sep_data, typesInfo, docsInfo = seperator(data)
# write(sep_data)

# dataFrameInitial = pd.DataFrame({"Type": typesInfo, "Document": docsInfo})

In [5]:
# dataFrameInitial.count()
# saveFrame(dataFrameInitial, "./raw_test_data.csv")

In [6]:
# load files
# load in docs you want to use:

## trained data info data set for use:
trainedFrame = load_objects("./raw_data_info_frame.pkl")

## vocabulary training set for use:
vocabulary = load_objects("./raw_vocabulary.pkl")

# test data
testFrame = pd.read_csv("./raw_test_data.csv", sep=",")

In [7]:
testFrame.head()

,Type,Document
0,atheism,re about the bible quiz answers in article hea...
1,atheism,re amusing atheists and agnostics in article t...
2,atheism,re yet more rushdie re islamic law jaeger buph...
3,atheism,re christian morality is in article vice ico t...
4,atheism,re after years can we say that christian moral...


In [8]:
trainedFrame.head()

,Type,Document,ClassProbability,WordPositions,WordCount
0,autos,re saturn s pricing policy in article c vir l ...,0.052599,127696,"{'akin': {'count': 0, 'probability': -12.21308..."
1,pc,wanted bus card for logitech mouse i ve acquir...,0.0522448,112583,"{'akin': {'count': 0, 'probability': -12.13508..."
2,mac,re nutek faces apple s wrath article read in a...,0.0511821,97182,"{'akin': {'count': 0, 'probability': -12.04879..."
3,forsale,motorcycle wanted followup to kedz wpi wpi edu...,0.051802,69761,"{'akin': {'count': 0, 'probability': -11.87390..."
4,baseball,spring records the orioles pitching staff agai...,0.0528646,119407,"{'akin': {'count': 0, 'probability': -12.17106..."


In [9]:
# create my result frame:
resultFrame = testFrame.copy()
resultFrame["Predicted"] = None
saveFrame(resultFrame, "./raw_result_frame")

In [10]:
# start predicting:
jobs = []
for index, row in testFrame.iterrows():
    unique_id = index
    print(unique_id)
    classProb = classDict(trainedFrame)
    
    thread = Thread(target = process, args = (trainedFrame, row, index, classProb, vocabulary, ))
    jobs.append(thread)
    break
    
# start the model threads
countS = 0
for job in jobs:
    print("Started: " + str(countS))
    countS = countS + 1
    job.start()
# wait for all threads to finish
countE = 0
for job in jobs:
    print("Ended:  " + str(countE))
    countE = countE + 1
    job.join()


0
Started: 0
Ended:  0


In [11]:
resultFrame.head()

,Type,Document,Predicted
0,atheism,re about the bible quiz answers in article hea...,"{'class': 'forsale', 'max': 3.3767820138852545..."
1,atheism,re amusing atheists and agnostics in article t...,None
2,atheism,re yet more rushdie re islamic law jaeger buph...,None
3,atheism,re christian morality is in article vice ico t...,None
4,atheism,re after years can we say that christian moral...,None
